### Data Preprocessing

In [17]:
import tiktoken
import torch
from torch.utils.data import Dataset,DataLoader

# get gpt2 encoder
tokenizer = tiktoken.get_encoding('gpt2')

# get raw text data
data_path = 'the-verdict.txt'
with open(data_path,'r',encoding='utf-8') as f:
    raw_text = f.read()

class MyDataset(Dataset):
    def __init__(self,raw_text,tokenizer,max_length,stride):
        """
        Dataset class of unlabeled text data.
        Create a dataset with a shifting window such that:
        target data is one right shifting of input data.

        Args:
            raw_text (str): raw text input
            tokenizer (tokenizer with encode method): tokenizer for encoding text data
            max_length (int): context window size 
            stride (int): shifting step
        """
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(raw_text,allowed_special={"<|endoftext|>"})
        for i in range(0,len(token_ids) - max_length,stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids[idx],self.target_ids[idx]

dataset = MyDataset(raw_text,tokenizer,max_length = 10,stride = 2)
dataloader = DataLoader(dataset,shuffle = False,batch_size = 2)
dataloader_iter = iter(dataloader)
inputs,targets = next(dataloader_iter)
print("sample inputs: ",inputs)
print("sample targets: ",targets)

sample inputs:  tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257,  7026]])
sample targets:  tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257],
        [ 1464,  1807,  3619,   402,   271, 10899,  2138,   257,  7026, 15632]])


In [53]:
input_text1 = tokenizer.decode(inputs[0].detach().numpy().tolist())
output_text1 = tokenizer.decode(targets[0].detach().numpy().tolist())
print(input_text1)
print(output_text1)
input_text2 = tokenizer.decode(inputs[1].detach().numpy().tolist())
output_text2 = tokenizer.decode(targets[1].detach().numpy().tolist())
print(input_text2)
print(output_text2)

I HAD always thought Jack Gisburn rather
 HAD always thought Jack Gisburn rather a
AD always thought Jack Gisburn rather a cheap
 always thought Jack Gisburn rather a cheap genius


### Model Architecture

In [18]:
import torch
import torch.nn as nn

GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

#### Word Embedding & Positional Embedding

Containing two parts, word embedding layer and positional embedding layer.

Word Embedding Layer is a trainable lookup table works as Word2Vec.

Positional Embedding is a trainable lookup table with input of positional index.

Output Dimension: (batch_size, num_tokens, embedding_dimension)

In [20]:
class TokenEmbedding(nn.Module):
    def __init__(self,vocab_size,emb_dim):
        super(TokenEmbedding, self).__init__()
        self.word_embedding = nn.Embedding(vocab_size,emb_dim)
        self.pos_embedding = nn.Embedding(vocab_size,emb_dim)
    
    def forward(self,x):
        _,num_tokens = x.shape
        word_embedding = self.word_embedding(x)
        pos_embedding = self.pos_embedding(torch.arange(0,num_tokens,device=x.device))
        return word_embedding + pos_embedding
    
embedding_layer = TokenEmbedding(GPT_CONFIG_124M["vocab_size"],GPT_CONFIG_124M["emb_dim"])
embedding_outputs = embedding_layer(inputs)
print(embedding_outputs.shape)

torch.Size([2, 10, 768])


#### Layer Normalization
Normalize the layer input with mean = 0 and std = 1, add additional trainable scaling and shifting parameters.

In LLM, LayerNorm works as the following:
Normalize each token vector in the sequence.
Does not change the dimension.

Output Dimension: (batch_size, num_tokens, embedding_dimension)

In [23]:
class LayerNorm(nn.Module):
    def __init__(self,emb_dim):
        super(LayerNorm,self).__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        
    def forward(self,x):
        mean = x.mean(dim=-1,keepdim=True)
        var = x.var(dim=-1,keepdim=True)
        norm_x = (x - mean) / torch.sqrt(var+self.eps)
        return self.scale * norm_x + self.shift
    
layer_norm = LayerNorm(GPT_CONFIG_124M['emb_dim'])
layer_norm_outputs = layer_norm(embedding_outputs)
print(layer_norm_outputs.shape)
        

torch.Size([2, 10, 768])


#### Dropout Layer

Normally this is set to 0.

In [25]:
class MyDropout(nn.Module):
    def __init__(self,dropout_rate):
        super(MyDropout,self).__init__()
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self,x):
        return self.dropout(x)
dropout_layer = MyDropout(GPT_CONFIG_124M["drop_rate"])
dropout_outputs = dropout_layer(layer_norm_outputs)
print(dropout_outputs.shape)

torch.Size([2, 10, 768])


#### Multihead Attention Layer

Output Dimension: (batch_size,num_tokens,embedding_dimension)

In [29]:
class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias=False):
        super(MultiHeadAttention,self).__init__()
        assert (d_out % num_heads == 0), "output dimension must be divisible by number of heads"
        self.d_in = d_in
        self.d_out = d_out
        self.num_head = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,bias=qkv_bias)
        self.out_proj = nn.Linear(d_out,d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask",torch.triu(torch.ones(context_length,context_length),diagonal=1))
        #registered mask is a very large mask matrix with HxW of maximum context length
        
    def forward(self,x):
        b,num_tokens,d_in = x.shape
        
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        
        queries = queries.view(b,num_tokens,self.num_head,self.head_dim)
        keys = keys.view(b,num_tokens,self.num_head,self.head_dim)
        values = values.view(b,num_tokens,self.num_head,self.head_dim)
        
        queries = queries.transpose(1,2)
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        
        attn_scores = torch.matmul(queries,keys.transpose(2,3))
        mask_bool = self.mask.bool()[:num_tokens,:num_tokens]
        attn_scores.masked_fill_(mask_bool,-torch.inf)
        attn_weights = torch.softmax(attn_scores/self.head_dim**0.5,dim=-1)
        attn_weights = self.dropout(attn_weights)
        context_vec = torch.matmul(attn_weights,values).transpose(1,2)
        context_vec = context_vec.contiguous().view(b,num_tokens,self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec
    
multi_head_attention_layer = MultiHeadAttention(GPT_CONFIG_124M["emb_dim"],GPT_CONFIG_124M["emb_dim"],GPT_CONFIG_124M["context_length"],GPT_CONFIG_124M["drop_rate"],GPT_CONFIG_124M["n_heads"])
mha_outputs = multi_head_attention_layer(dropout_outputs)
print(mha_outputs.shape)

torch.Size([2, 10, 768])


#### Feed Forward Layer

In [38]:
class FeedForward(nn.Module):
    def __init__(self,d_in,d_out):
        super(FeedForward,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(d_in,d_in*4),
            nn.GELU(),
            nn.Linear(4*d_in,d_out)
        )
    
    def forward(self,x):
        return self.layers(x)
ff_layer = FeedForward(GPT_CONFIG_124M["emb_dim"],GPT_CONFIG_124M["emb_dim"])
ff_layer_outputs = ff_layer(mha_outputs)
print(ff_layer_outputs.shape)

torch.Size([2, 10, 768])


### Output Head Layer

Project output dimension to Vocab

In [41]:
output_head = nn.Linear(GPT_CONFIG_124M['emb_dim'],GPT_CONFIG_124M['vocab_size'])
output_logits = output_head(ff_layer_outputs)
print(output_logits.shape)

torch.Size([2, 10, 50257])


#### Generating Text

In [63]:
logits = output_logits[:,-1,:]
idx_next = torch.argmax(logits,dim=-1,keepdim=True)
result = torch.cat([inputs,idx_next],dim=1)

In [66]:
result_text1 = tokenizer.decode(result[0].detach().numpy().tolist())
result_text2 = tokenizer.decode(result[1].detach().numpy().tolist())
print(result_text1)
print(result_text2)

I HAD always thought Jack Gisburn rather Shoes
AD always thought Jack Gisburn rather a cheapabul


### Put things together

In [73]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg['emb_dim'],cfg['emb_dim'])
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x
    
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


In [72]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)

out = model(inputs)
print("Input batch:\n", inputs)
print("\nOutput shape:", out.shape)
print(out)

Input batch:
 tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257,  7026]])

Output shape: torch.Size([2, 10, 50257])
tensor([[[ 0.7372, -0.2461,  0.7377,  ..., -0.5505,  1.0940, -0.1215],
         [ 0.1349, -0.4291,  0.2044,  ..., -0.9043, -0.6264,  0.3131],
         [ 0.1940,  0.3353,  0.7451,  ...,  0.3835, -1.0765,  0.0152],
         ...,
         [-0.2486, -0.6441, -0.0191,  ..., -0.8061, -0.4295, -0.5307],
         [ 0.1751, -0.2373,  1.1876,  ..., -0.7559, -0.1669, -0.0469],
         [-0.3735,  0.1831,  0.7372,  ...,  0.0771, -0.2125,  0.0353]],

        [[-0.6498,  0.2321,  0.7633,  ...,  0.7106, -0.4916, -0.9599],
         [ 0.6222,  0.3213, -0.1542,  ...,  0.5888,  0.7959, -1.2487],
         [ 1.1046, -0.0211,  0.7930,  ...,  0.1300, -0.3611, -0.4926],
         ...,
         [ 0.2321, -0.2643,  0.4346,  ..., -0.6386, -0.2357, -0.5369],
         [ 1.2384,  0.3239,  0.7638,  ..., 

In [76]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]  

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

model.eval() # disable dropout

out = generate_text_simple(
    model=model,
    idx=inputs, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output:", out)
print("Output length:", len(out[0]))
decoded_text = tokenizer.decode(out[0].squeeze(0).tolist())
print(decoded_text)

Output: tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,
         19530, 43231, 30967, 42175, 48044, 44052],
        [ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257,  7026,
         26243, 24810, 48803, 49703, 40506, 30168]])
Output length: 16
I HAD always thought Jack Gisburn rather Forward Normandy freezerigrate McAuliffe expr


### Check model outputs before training

In [104]:
with torch.no_grad():
    logits = model(inputs)
#print("model outputs: ",logits)
probas = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
#print(probas.shape) # Shape: (batch_size, num_tokens, vocab_size)
token_ids = torch.argmax(probas,dim=-1,keepdim=True)
#print("Token IDs:\n", token_ids)
print("Target: ",tokenizer.decode(inputs[0].flatten().squeeze(dim=0).tolist()))
print("Outputs: ",tokenizer.decode(token_ids[0].flatten().squeeze(dim=0).tolist()))

Target:  I HAD always thought Jack Gisburn rather
Outputs:   cosickson temples wast discriminatedFairredo conjectureometer Forward


### Loss Backward:

pytorch nn.functional.cross_entropy take cares of everything

In [93]:
print(logits.shape)
print(targets.shape)

torch.Size([2, 10, 50257])
torch.Size([2, 10])


#### Flatten batch with sequence, aka puts batches together and calculate batch loss, loss of individual batch is batch loss/num_batch

In [102]:
flatten_batch_logits = logits.flatten(0,1)
flatten_batch_targets = targets.flatten(0,1)
print(flatten_batch_logits.shape)
print(flatten_batch_targets.shape)

torch.Size([20, 50257])
torch.Size([20])


#### nn.function.cross_entory

nn.functional.cross_entropy(input=logits,target=integers)
- logits has dimension of vocab
- target has index on the dimension

In [103]:
loss = nn.functional.cross_entropy(flatten_batch_logits,flatten_batch_targets)
loss

tensor(11.0605)

In [105]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

In [108]:
def cal_batch_loss(input_batch,target_batch,model,device):
    input_batch,target_batch = input_batch.to(device),target_batch.to(device)
    logits = model(input_batch)
    loss = nn.functional.cross_entropy(logits.flatten(0,1),target_batch.flatten())
    return loss

def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = cal_batch_loss(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
        decoded_text = token_ids_to_text(token_ids, tokenizer)
        print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

def train(model,train_loader,val_loader,optimizer,device,num_epochs,eval_freq,eval_iter,start_context,tokenizer):
    train_losses, val_losses, track_tokens_seen = [],[],[]
    tokens_seen = 0
    global_step = -1
    for epoch in range(num_epochs):
        model.train()
        for input_batch,target_batch in train_loader:
            optimizer.zero_grad()
            loss = cal_batch_loss(input_batch,target_batch,model,device)
            loss.backward()
            optimizer.step()
            tokens_seen += input_batch.numel()
            global_step += 1      
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(model,train_loader,val_loader,device,eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )
    return train_losses,val_losses,track_tokens_seen

In [109]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train(
    model, dataloader, dataloader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

UnboundLocalError: cannot access local variable 'token_seen' where it is not associated with a value